# You have taken your videos, but what REALLY is going on in your samples?

Rather than manually track how your beads, DNA samples, or active particles are moving, we are going to use a nifty technique called Differential Dynamic Microscopy to exract important the information. 

So, let's begin. Load in all of the python modules in the following cell. (It's ok for the red warning signs to appear...It will still run!)

In [17]:
import numpy as np
import matplotlib
%matplotlib notebook
from matplotlib import pyplot as plt

from IPython.html.widgets import interact, fixed
import ipywidgets as widgets

import io 

font_plt = {'family': 'serif',
            'color':  'darkred',
            'weight': 'normal',
            'size': 10,
            }
font_plt_ax = {'family': 'serif',
               'color':  'black',
               'weight': 'normal',
               'size': 10,
              }
import sys
import glob #glob is helpful for searching for filenames or directories
import ddm_clean as ddm #this is the module containing the differential dynamic microscopy code
import tiff_file
import scipy #scientific python
import pickle #for saving data

In [18]:
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt


Load in the data directory. I like to have a separate folder for just .tif files, to keep things more organized. You can do this or a different system. 

The function glob.glob will pull out files with the specific qualities that you require. Here we want files ending with 'ome.tif' so we add the ** in front of 'ome.tif' to mark that there can be anything before the ome.tif. 

If there are multiple files they will then be sorted into the list named files.

In [38]:
Data_dir = "Z:\\KiraTommy_Summer2019\\Data\\2019-06-11_1to10_100nm_forDDM\\Video_20%_1\\"
files = glob.glob(Data_dir+"*tif")
files.sort()
print "Files founds: ", files

Files founds:  ['Z:\\KiraTommy_Summer2019\\Data\\2019-06-11_1to10_100nm_forDDM\\Video_20%_1\\Video_20%_1_MMStack_Pos0.ome.tif', 'Z:\\KiraTommy_Summer2019\\Data\\2019-06-11_1to10_100nm_forDDM\\Video_20%_1\\Video_20%_1_MMStack_Pos0_2x2bin.tif']


Next we will set a few parameters and limits for our analysis.

You will have taken videos with upwards of 1000 frames (for DNA videos they are perferably 3000+). These videos can be broken up into groups of different frame differences. We can control what the frame differences are by changing the endframelag variable. 

We have mostly used 998, 1998, and 2998 depending on the number of frames in the video. For more than 3000 frames I tend to use 2998. 

We can also change the number of values sampled out of those different time lags (frame differences). Usually we choose 400, but for a quick overview of how the code work we can stick with 20. 

In [35]:
endframelag = 500
numvaluessampled = 20

dts = ddm.genLogDist(1,endframelag,numvaluessampled)


Now we are going to set the region of interest we are going to analyze. FFTs work well with dimensions of 2^n, so keep that in mind when choosing your region of interest (ROI). 

For DNA videos we have been doing 256x256 px ROIs, usually the videos are 1280x256 so we get 5 ROIs per video.

In the cell below, label your ROI in the variable ROI (if you were to do more ROIs you can label them in a list form)

In [39]:
file_to_analyze = files[0]
print "Analyzing file: %s" % file_to_analyze
im = tiff_file.imread(file_to_analyze)

Analyzing file: Z:\KiraTommy_Summer2019\Data\2019-06-11_1to10_100nm_forDDM\Video_20%_1\Video_20%_1_MMStack_Pos0.ome.tif


In [40]:
start_frame = 0 #
fft,dtsB = ddm.getFFTDiffsAtTimes(im[start_frame:,:,:], dts, every=None, limitImsTo=None, submean=True)
ravs = ddm.radialAvFFTs_v2(fft)
dts_fft = {'dts':dts, 'ffts':fft, 'ravs':ravs}
pickle.dump(dts_fft, open(file_to_analyze+"_512x512_FFTDIFFS_dts_ravs.p",'wb'))

MemoryError: 

Only one ROI: run the cell below

There is a separate version (two cells down) if you have multiple ROIs to run

In [ ]:
ROI_v = 'Center'

ran_v = 256

ran2_v = 512

# you will want to change the initial y and end y values depending on the size of your video and where you want your ROI to be.
inty_value = 5

endy_value = 261

Now, the actual meat of the code! This for loop will run through all of the videos in the list named files (from earlier cell). This will only analyze one ROI and will do so for the region you created above.

All of the FFT files will be saved to the file where the videos are taken from, but you can move them after the code has finished. 

This code can take a while to run, especially if the numvaluessampled variable is 400. 

Run the next cell to truly begin analysis

In [ ]:
for i in range(len(files)):
    
    im = tiff_file.imread(files)
    fft,dtsB = ddm_clean.getFFTDiffsAtTimes(im[:,inty_value:endy_value,ran_v:ran2_v], dts, every=None, limitImsTo=None, submean=True)
    ravs = ddm_clean.radialAvFFTs_v2(fft)
    dts_fft = {'dts':dts, 'ffts':fft, 'ravs':ravs}
    pickle.dump(dts_fft, open(files[i][:-4]+"_"+ROI_v+"_128x128_FFTDIFFS_dts_ravs.p",'wb'))

Multiple ROIs: run the cell below 

The cell following the next is the same as the one above, but has a second for loop to run through the multiple ROIs. 

The y values do not need to change. 

In [ ]:
ROI = ['0', '128', '256', '384', '512', '640', '768', '896']

ran = [0, 128, 256, 384, 512, 640, 768, 896]

ran2 = [128, 256, 384, 512, 640, 768, 896, 1024]

# you will want to change the initial y and end y values depending on the size of your video and where you want your ROI to be.
inty_value = 5

endy_value = 261

In [ ]:
for i in range(len(files)):
    
    im = tiff_file.imread(files)
    
    for j in range(len(ROI)):
        
        fft,dtsB = ddm_clean.getFFTDiffsAtTimes(im[:,5:133,ran[j]:ran2[j]], dts, every=None, limitImsTo=None, submean=True)
        ravs = ddm_clean.radialAvFFTs_v2(fft)
        dts_fft = {'dts':dts, 'ffts':fft, 'ravs':ravs}
        pickle.dump(dts_fft, open(files[i][:-4]+"_"+ROI[j]+"_128x128_FFTDIFFS_dts_ravs.p",'wb'))

Once this code has finished running continue on to Step 2. DDM fitting ravs to find taus